## THOR Tutorials - Initial Orbit Determination

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time

from astroquery.jplhorizons import Horizons

from thor.orbits.ephemeris import generateEphemeris
from thor.orbits.iod import iod

/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:55: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  e_vec = ((v_mag**2 - mu / r_mag) * r - (np.dot(r, v)) * v) / mu
/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:77: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, C), array(float64, 1d, A))
  trueAnom_deg = np.degrees(np.arccos(np.dot(e_vec, r) / (e * r_mag)))
/Users/moeyensj/projects/thor/thor/thor/orbits/kepler.py:78: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  if np.dot(r, v) < 0:
/Users/moeyensj/projects/thor/thor/thor/orbits/propagate/universal.py:46: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  rv_mag = np.dot(r, v) / r_mag


Like we did in the ephemeris generation notebook, we first need to create some observations which we can then use to run initial orbit determination. The first few cells in this notebook repeat what we did in the ephemeris generation notebook.

As before, let us first get some orbits from JPL's Horizons service. 

Create a list of your favorite minor planets:

In [2]:
targets = [
    "Ceres",
    "Eros",
    "Duende",
    "Amor",
    "Aten"
]

We will query Horizons for state vectors only for a single epoch but once we have those state vectors, we can then generate ephemeris for the remaining 'observation' times.

In [3]:
start_mjd = 57580.0
end_mjd = start_mjd + 14.0
step_mjd = 0.5

observation_times = Time(
    np.arange(start_mjd, end_mjd + step_mjd, step_mjd),
    scale="utc",
    format="mjd"
)

In [4]:
observation_times[0].isot

'2016-07-11T00:00:00.000'

In [5]:
orbits = []
t0 = []
for name in targets:
    obj = Horizons(id=name, epochs=observation_times[0], location="@sun")
    df = obj.vectors().to_pandas()
    orbit = df[["x", "y", "z", "vx", "vy", "vz"]].values
    orbits.append(orbit[0, :])
    
    t0.append(observation_times[0])
    
orbits = np.array(orbits)
t0 = Time(t0)

Now we have an array of cartesian state vectors and an array of epochs at which those state vectors are defined. 

In [6]:
orbits

array([[ 2.86107593e+00,  4.05024279e-01, -5.14638072e-01,
        -1.70128971e-03,  9.56388437e-03,  6.14460690e-04],
       [ 1.22114897e+00, -1.28550678e+00,  5.42153575e-02,
         7.67767895e-03,  8.20474710e-03,  2.09775644e-03],
       [-4.87939838e-01,  8.29808004e-01, -8.82658763e-02,
        -1.50320455e-02, -7.25226377e-03,  2.93262913e-03],
       [ 1.92186805e+00,  1.68045745e+00, -4.10309701e-01,
        -7.34528933e-03,  4.50055820e-03, -7.03468368e-04],
       [ 9.37710396e-01,  5.85051521e-02, -3.11307133e-01,
         1.44880779e-03,  1.68644847e-02, -2.31265764e-03]])

In [7]:
t0

<Time object: scale='utc' format='mjd' value=[57580. 57580. 57580. 57580. 57580.]>

The last remaining component we need is to designated a few observers.

In [31]:
observers = {
    "000" : observation_times[1:] - 5/24.,
    "I11" : observation_times[1:],
    "I41" : observation_times[1:] + 1/24.
}

We now have everything we need to generate ephemeris for each object. 

In [32]:
ephemeris = generateEphemeris(
    orbits,
    t0,
    observers
)

In [33]:
ephemeris

,orbit_id,observatory_code,mjd_utc,RA_deg,Dec_deg,vRAcosDec,vDec,r_au,delta_au,light_time,...,obj_z,obj_vx,obj_vy,obj_vz,obs_x,obs_y,obs_z,obs_vx,obs_vy,obs_vz
0,0,000,57580.291667,30.015381,1.413682,0.229313,0.048661,2.934950,2.918712,0.016857,...,-0.514469,-0.001710,0.009563,0.000616,0.334089,-0.960138,0.000057,0.015908,0.005675,-0.000035
1,0,000,57580.791667,30.130802,1.437795,0.232014,0.047884,2.934726,2.911986,0.016818,...,-0.514160,-0.001727,0.009560,0.000619,0.342017,-0.957335,0.000069,0.015979,0.005652,0.000033
2,0,000,57581.291667,30.245349,1.461567,0.226503,0.047082,2.934502,2.905147,0.016779,...,-0.513850,-0.001744,0.009558,0.000622,0.350008,-0.954408,0.000056,0.015809,0.005943,-0.000035
3,0,000,57581.791667,30.359377,1.484889,0.229196,0.046303,2.934277,2.898408,0.016740,...,-0.513538,-0.001761,0.009555,0.000625,0.357887,-0.951470,0.000068,0.015881,0.005921,0.000033
4,0,000,57582.291667,30.472491,1.507870,0.223647,0.045494,2.934052,2.891558,0.016700,...,-0.513225,-0.001777,0.009553,0.000628,0.365827,-0.948409,0.000055,0.015706,0.006209,-0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,4,I41,57592.041667,70.258224,6.703400,0.830520,0.080026,1.025601,1.255529,0.007251,...,-0.332334,-0.001821,0.016317,-0.001190,0.513915,-0.876302,0.000056,0.014619,0.008470,0.000071
416,4,I41,57592.541667,70.673049,6.743830,0.818587,0.080726,1.027019,1.256357,0.007256,...,-0.332917,-0.001949,0.016281,-0.001144,0.521243,-0.871933,0.000047,0.014422,0.008919,-0.000070
417,4,I41,57593.041667,71.082226,6.783344,0.828308,0.078313,1.028430,1.257251,0.007261,...,-0.333478,-0.002076,0.016244,-0.001099,0.528400,-0.867546,0.000057,0.014477,0.008717,0.000070
418,4,I41,57593.541667,71.495936,6.822922,0.816371,0.079013,1.029836,1.258034,0.007266,...,-0.334016,-0.002203,0.016206,-0.001053,0.535654,-0.863053,0.000047,0.014272,0.009162,-0.000070


We now have a data frame of simulated observations. Let's add a few columns to represent astrometric errors and a column for observation IDs which we will need to run IOD. Then let's also drop the orbit_id column (our ground truth) and a few other columns so we get as barebones of a data frame as needed by THOR. 

In [34]:
ephemeris.insert(0, "obs_id", np.arange(1, len(ephemeris) + 1, dtype=int))

observations = ephemeris.copy()
observations.insert(6, "RA_sigma_deg", [0.1/3600 for i in range(len(observations))])
observations.insert(7, "Dec_sigma_deg", [0.1/3600 for i in range(len(observations))])

observations.drop(
    columns=[
        'orbit_id', 'vRAcosDec', 'vDec', 'r_au', 'delta_au', 'light_time', 'obj_x', 'obj_y',
        'obj_z', 'obj_vx', 'obj_vy', 'obj_vz',
        'obs_vx', 'obs_vy', 'obs_vz'
    ],
    inplace=True
)

In [35]:
observations

,obs_id,observatory_code,mjd_utc,RA_deg,Dec_deg,RA_sigma_deg,Dec_sigma_deg,obs_x,obs_y,obs_z
0,1,000,57580.291667,30.015381,1.413682,0.000028,0.000028,0.334089,-0.960138,0.000057
1,2,000,57580.791667,30.130802,1.437795,0.000028,0.000028,0.342017,-0.957335,0.000069
2,3,000,57581.291667,30.245349,1.461567,0.000028,0.000028,0.350008,-0.954408,0.000056
3,4,000,57581.791667,30.359377,1.484889,0.000028,0.000028,0.357887,-0.951470,0.000068
4,5,000,57582.291667,30.472491,1.507870,0.000028,0.000028,0.365827,-0.948409,0.000055
...,...,...,...,...,...,...,...,...,...,...
415,416,I41,57592.041667,70.258224,6.703400,0.000028,0.000028,0.513915,-0.876302,0.000056
416,417,I41,57592.541667,70.673049,6.743830,0.000028,0.000028,0.521243,-0.871933,0.000047
417,418,I41,57593.041667,71.082226,6.783344,0.000028,0.000028,0.528400,-0.867546,0.000057
418,419,I41,57593.541667,71.495936,6.822922,0.000028,0.000028,0.535654,-0.863053,0.000047


The plan for this notebook is not to run a linking algorithm to create possible linkages out of these observations. So let us instead assume that we have run THOR (or similar) and that it has returned to use a data frame describing a few viable linkages. 

In [51]:
linkage_members = []
num_obs = 10
linkage_id = 0
# Add one pure linkage for each orbit 
for i, orbit_id in enumerate(ephemeris["orbit_id"].unique()):
    obs_ids = np.random.choice(ephemeris[ephemeris["orbit_id"] == orbit_id]["obs_id"].values, num_obs, replace=False)
    
    linkage_id += 1
    linkage_members.append(
        pd.DataFrame(
            np.vstack([
                np.array([linkage_id for j in range(len(obs_ids))]),
                obs_ids]
            ).T,
            columns=[
                "linkage_id",
                "obs_id"
            ]
        )
    )

# Add in a few partial linkages (mixed detections but  still discoverable)
for i, orbit_id in enumerate(ephemeris["orbit_id"].unique()):
    obs_ids = np.random.choice(ephemeris[ephemeris["orbit_id"] == orbit_id]["obs_id"].values, num_obs, replace=False)
    obs_ids_other = np.random.choice(ephemeris[ephemeris["orbit_id"] != orbit_id]["obs_id"].values, 1, replace=False)
    
    obs_ids = np.concatenate([obs_ids, obs_ids_other])
    linkage_id += 1
    linkage_members.append(
        pd.DataFrame(
            np.vstack([
                np.array([linkage_id for j in range(len(obs_ids))]),
                obs_ids]
            ).T,
            columns=[
                "linkage_id",
                "obs_id"
            ]
        )
    )
    
# Lastly, add in a few mixed linkages: just junk! 
for i, orbit_id in enumerate(ephemeris["orbit_id"].unique()):
    obs_ids = np.random.choice(ephemeris[ephemeris["orbit_id"] == orbit_id]["obs_id"].values, 2, replace=False)
    obs_ids_other = np.random.choice(ephemeris[ephemeris["orbit_id"] != orbit_id]["obs_id"].values, num_obs - 2, replace=False)
    
    obs_ids = np.concatenate([obs_ids, obs_ids_other])
    linkage_id += 1
    linkage_members.append(
        pd.DataFrame(
            np.vstack([
                np.array([linkage_id for j in range(len(obs_ids))]),
                obs_ids]
            ).T,
            columns=[
                "linkage_id",
                "obs_id"
            ]
        )
    )

linkage_members = pd.concat(linkage_members)
linkage_members.sort_values(by=["linkage_id", "obs_id"], inplace=True)
linkage_members.reset_index(inplace=True, drop=True)


In [53]:
linkage_members.head(14)

,linkage_id,obs_id
0,1,7
1,1,12
2,1,141
3,1,142
4,1,145
5,1,146
6,1,159
7,1,293
8,1,302
9,1,306


We now need to define a dictionary which maps internal columns to external columns:

In [54]:
columnMapping = {
    "obs_id" : "obs_id",
    "exp_mjd" : "mjd_utc",
    "RA_deg" : "RA_deg",
    "Dec_deg" : "Dec_deg",
    "RA_sigma_deg" : "RA_sigma_deg",
    "Dec_sigma_deg" : "Dec_sigma_deg",
    "observatory_code" : "observatory_code",
    "obs_x_au" : "obs_x",
    "obs_y_au" : "obs_y", 
    "obs_z_au" : "obs_z"
}

In [63]:
linkage_id = 1
obs_ids = linkage_members[linkage_members["linkage_id"] == linkage_id]["obs_id"].values

orbit, orbit_members, outliers = iod(observations[observations["obs_id"].isin(obs_ids)], 
                                     observation_selection_method="combinations", 
                                     chi2_threshold=1000, 
                                     contamination_percentage=20.0, 
                                     column_mapping=columnMapping
                                    )

In [64]:
orbit

,orbit_id,epoch_mjd_utc,obj_x,obj_y,obj_z,obj_vx,obj_vy,obj_vz,arc_length,num_obs,chi2
0,141_293_306_v3,57586.525303,2.849018,0.467185,-0.510449,-0.00192,0.009531,0.000654,12.541666666664241,10,205.334676


In [65]:
outliers

array([], dtype=float64)

In [66]:
orbit_members["obs_id"].values

array(['7', '12', '141', '142', '145', '146', '159', '293', '302', '306'],
      dtype=object)

In [67]:
observations[observations["obs_id"].isin(orbit_members["obs_id"].values)]

,obs_id,observatory_code,mjd_utc,RA_deg,Dec_deg,RA_sigma_deg,Dec_sigma_deg,obs_x,obs_y,obs_z
6,7,000,57583.291667,30.696761,1.552582,0.000028,0.000028,0.381542,-0.942144,0.000055
11,12,000,57585.791667,31.244723,1.657313,0.000028,0.000028,0.420304,-0.925360,0.000067
140,141,I11,57580.500000,30.063451,1.424865,0.000028,0.000028,0.337421,-0.958981,0.000004
141,142,I11,57581.000000,30.178725,1.448805,0.000028,0.000028,0.345325,-0.956164,0.000022
144,145,I11,57582.500000,30.519376,1.518403,0.000028,0.000028,0.369116,-0.947140,0.000002
145,146,I11,57583.000000,30.631843,1.540755,0.000028,0.000028,0.376920,-0.944057,0.000021
158,159,I11,57589.500000,32.021809,1.795172,0.000028,0.000028,0.476521,-0.897424,-0.000001
292,293,I41,57586.541667,31.405538,1.687010,0.000028,0.000028,0.431858,-0.919978,0.000047
301,302,I41,57591.041667,32.332082,1.844374,0.000028,0.000028,0.499286,-0.884811,0.000056
305,306,I41,57593.041667,32.722778,1.903537,0.000028,0.000028,0.528400,-0.867546,0.000057
